In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [ ]:
def reoccur():
    repeat = input("Enter y or Y for calculating again: ")
    if(repeat == 'y' or repeat == 'Y'):
        main()     
    else:
        print("Thank you for using this calculator")
def main():
    options_provided = {1:"self weight of beam", 2:"total and factorial load", 3:"design moment", 4:"design shear force", 5:"effective cover", 6:"effective depth of beam", 7:"limiting moment resistance", 8:"Asc for doubly reinforced beam", 9:"compression bars required",10:"Ast for doubly reinforced beam", 11:"torsion bars required", 12:"development length"}
    options = int(input(f"Choose one options from following:\n{options_provided}: "))

    # self weight
    if(options==1):
        overall_depth_beam = float(input("Enter the depth of beam in mm: "))
        width_beam = float(input("Enter the width of beam in mm: "))
        self_weight_beam = 25* (overall_depth_beam/1000) * (width_beam/1000)
        print(f"self weight of beam is {self_weight_beam}kN/m")
    # total and factorial load
    elif(options==2):
        live_load = float(input("Enter live load in kN/m: "))
        imposed_load = float(input("Enter imposed load in kN/m: "))
        floor_finish = float(input("Enter floor finish in kN/m: "))
        self_weight_beam = float(input("Enter self weight of beam you obtained in kN/m: "))
        total_load = live_load + imposed_load + floor_finish + self_weight_beam
        factored_load = 1.5* total_load
        print(f"Total load is {total_load}kN/m and factored load is {factored_load}kN/m")
    #design moment
    elif(options==3):
        beam_span = float(input("Enter beam span in  meter: "))
        factored_load = float(input("Enter the factored load you obtained before in kN/m: "))
        design_moment = (factored_load * (beam_span**2))/8
        print(f"design moment of beam is {design_moment}kNm")
    #design shear force
    elif(options==4):
        beam_span = float(input("Enter beam span in  meter: "))
        factored_load = float(input("Enter the factored load you obtained before in kN/m: "))
        design_shear_force = (factored_load * beam_span)/2
        print(f"Design Shear Force is equal to {design_shear_force}kN")
    #effective cover
    elif(options==5):
        table4_4 = pd.read_excel('Table 4_4.xlsx')
        fck = int(input("Enter the value of concrete grade(20, 25, 30, 35, 40): "))
        effective_cover = table4_4.iloc[0,int(fck/5)-3]
        print(f"Effective cover is {effective_cover}mm")
    #effective depth of beam
    elif(options==6):
        overall_depth_beam = float(input("Enter the depth of beam in mm: "))
        effective_cover = float(input("Enter the effective cover you obtained in mm: "))
        effective_depth_beam = overall_depth_beam - effective_cover
        print(f"Effective depth of beam after considering effective cover is {effective_depth_beam}mm")

    #limiting moment resistance
    elif(options==7):
        width_beam = float(input("Enter the width of beam(mm): "))
        fy = int(input("Enter the value of steel grade(250,415,500): "))
        fck = int(input("Enter the value of concrete grade(20, 25, 30, 35, 40): "))
        effective_depth_beam = float(input("Enter Effective depth of beam after considering effective cover that you obtained earlier in mm: "))
        if (fy==250):
            x_u = 0.53 * (effective_depth_beam/1000)
        elif (fy==415):
            x_u = 0.48 * (effective_depth_beam/1000)
        else:
            x_u = 0.46 * (effective_depth_beam/1000)
        limiting_moment_resistance = 0.36 * fck * (width_beam) * x_u * ((effective_depth_beam/1000) - (x_u *0.42))
        print(f"Limiting  moment resistance is {limiting_moment_resistance}")
    #Asc for doubly reinforced beam
    elif(options==8):
        table5_1 = pd.read_excel('Table 5_1.xlsx')
        effective_depth_beam = float(input("Enter Effective depth of beam after considering effective cover that you obtained earlier in mm: "))
        effective_cover = float(input("Enter the effective cover you obtained in mm: "))
        fy = int(input("Enter the value of steel grade(250,415,500): "))
        limiting_moment_resistance = float(input("Enter the limiting moment resistance which obtained above: "))
        design_moment = float(input("Enter the design moment in kNm which you obtained earlier: "))
        if (fy==250):
            x_u = 0.53 * (effective_depth_beam/1000)
        elif (fy==415):
            x_u = 0.48 * (effective_depth_beam/1000)
        else:
            x_u = 0.46 * (effective_depth_beam/1000)
        index_temp_y = int(np.floor(((effective_cover/ effective_depth_beam)*100)/5))
        if ((effective_cover/ effective_depth_beam)<0.05):
            fsc =min((( 0.0035 * (x_u * 1000 - effective_cover)*200000)/(x_u * 1000)), 0.87*fy)
        else:
            if (fy == 250):
                fsc = ((table5_1.iloc[1,index_temp_y] + (((effective_cover/ effective_depth_beam)-table5_1.iloc[0,index_temp_y])*(0.5/(table5_1.iloc[1,index_temp_y +1] - table5_1.iloc[1,index_temp_y])))),0.87*fy)

            elif (fy == 415):
                fsc = ((table5_1.iloc[2,index_temp_y] + (((effective_cover/ effective_depth_beam)-table5_1.iloc[0,index_temp_y])*(0.5/(table5_1.iloc[2,index_temp_y +1] - table5_1.iloc[2,index_temp_y])))),0.87*fy)

            else:
                fsc = ((table5_1.iloc[3,index_temp_y] + (((effective_cover/ effective_depth_beam)-table5_1.iloc[0,index_temp_y])*(0.5/(table5_1.iloc[3,index_temp_y +1] - table5_1.iloc[3,index_temp_y])))),0.87*fy)

        Asc = ((design_moment - limiting_moment_resistance)/(fsc * (effective_depth_beam - effective_cover)))*(10**6)
        print(f"Asc is {Asc}sq.mm")
    # compression bars required
    elif(options==9):
        Asc = float(input("Enter the value of Asc in sq.mm which obtained before: "))
        diameter_rod_compression = int(input("Enter the diameter of compression bar you want to take in mm[8,10,12,14,16,18,20,22,25,28,32,40]: "))
        number_bars_compression = np.ceil((Asc/((np.pi)* (1/4) * (diameter_rod_compression **2))))
        print(f"you need {number_bars_compression} compresssion bars of diameter {diameter_rod_compression}")
    #Ast for doubly reinforced beam 
    elif(options==10):
        effective_depth_beam = float(input("Enter Effective depth of beam after considering effective cover that you obtained earlier in mm: "))
        fy = int(input("Enter the value of steel grade(250,415,500): "))
        design_moment = float(input("Enter the design moment in kNm which you obtained earlier: "))
        if (fy==250):
            x_u = 0.53 * (effective_depth_beam/1000)
        elif (fy==415):
            x_u = 0.48 * (effective_depth_beam/1000)
        else:
            x_u = 0.46 * (effective_depth_beam/1000)
        Ast = (design_moment/(0.87*fy*(effective_depth_beam - 0.42*(x_u*1000))))*(10**6)
        print(f"Ast is {Ast}sq.mm")
    #torsion bars required
    elif(options==11):
        Ast = float(input("Enter the value of Ast in sq.mm which obtained before: "))
        diameter_rod_tension = int(input("Enter the the diameter of bar you want to take in mm[8,10,12,14,16,18,20,22,25,28,32,40]: "))
        number_bars_tension = np.ceil((Ast/((np.pi)* (1/4) * (diameter_rod_tension **2))))
        print(f"you need {number_bars_tension} tension bars of diameter {diameter_rod_tension}")
    #development length
    else:
        clause26_2_1_1 = pd.read_excel('clause 26_2_1_1.xlsx')
        diameter_rod_compression = int(input("Enter the diameter of compression bar you want to take in mm[8,10,12,14,16,18,20,22,25,28,32,40]: "))
        diameter_rod_tension = int(input("Enter the the diameter of bar you want to take in mm[8,10,12,14,16,18,20,22,25,28,32,40]: "))
        fy = int(input("Enter the value of steel grade(250,415,500): "))
        fck = int(input("Enter the value of concrete grade(20, 25, 30, 35, 40): "))
        development_length_compression = (diameter_rod_compression * 0.87*fy)/(4*1.6*clause26_2_1_1.iloc[0,int ((fck/5)-3)])
        development_length_tension = (diameter_rod_tension * 0.87*fy)/(4*1.6*clause26_2_1_1.iloc[0,int((fck/5)-3)])
        print(f"Development length compression in mm {development_length_compression}")
        print(f"Devlopment length tension in mm {development_length_tension}")
    reoccur()
main()
